In [10]:
import numpy as np 
import matplotlib.pyplot as plt 

def get_sin_cos(theta):
    return ( np.sin(theta), np.cos(theta))

def rad2deg(rad):
    return np.array(rad) * (180.0/np.pi)

def deg2rad(deg):
    return np.array(deg) * (np.pi/180.0)

def rotationX(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[1,0,0,],
                    [0,  cos_theta, sin_theta ],
                    [0, -sin_theta, cos_theta ]])

def rotationY(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[cos_theta,0, -sin_theta],
                    [0, 1, 0 ],
                    [sin_theta, 0, cos_theta ]])

def rotationZ(theta):
    sin_theta, cos_theta = get_sin_cos(theta)
    return np.array([[cos_theta , sin_theta, 0  ],
                    [-sin_theta , cos_theta, 0  ],
                    [0, 0, 1 ]])

def dcmFromEulerXYZ(attitude_xyz):
    rx = rotationX(attitude_xyz[0])
    ry = rotationY(attitude_xyz[1])
    rz = rotationZ(attitude_xyz[2])
    return np.matmul(rx, np.matmul(ry,rz))


def printQuaternion(title, quat):
    print(f"Quaternion {title} [{quat[0]},  {quat[1]},  {quat[2]}, {quat[3]}]")



In [14]:
def quatNorm(quat):
    q0 = quat[0]
    q1 = quat[1]
    q2 = quat[2]
    q3 = quat[3]
    return np.sqrt(q0 * q0 + q1 * q1 + q2 * q2 + q3 * q3)

def conjugate(quat):
    q0 = quat[0]
    q1 = quat[1] # imaginary part
    q2 = quat[2] # imaginary part
    q3 = quat[3] # imaginary part
    return np.array([q0,-q1,-q2,-q3])

def quaternionInv(quat):
    conj = conjugate(quat)
    norm = quatNorm(quat)
    return conj/norm

def quaternionMult(quat1, quat2):
    q0 = quat1[0]
    q1 = quat1[1]
    q2 = quat1[2]
    q3 = quat1[3]
    Q = np.array([[ q0, -q1, -q2, -q3 ],
                  [ q1,  q0,  q3, -q2 ],
                  [ q2, -q3,  q0,  q1 ],
                  [ q3,  q2, -q1,  q0 ]])
    return np.matmul(Q, quat2)

def quaternionFromDCM(dcm):
    q1_den = ( 1.0 + dcm[0][0] + dcm[1][1] + dcm[2][2])
    q2_den = ( 1.0 + dcm[0][0] - dcm[1][1] - dcm[2][2])
    q3_den = ( 1.0 - dcm[0][0] + dcm[1][1] - dcm[2][2])
    q4_den = ( 1.0 - dcm[0][0] - dcm[1][1] + dcm[2][2])
    r23mr32 = dcm[1][2] - dcm[2][1]
    r31mr13 = dcm[2][0] - dcm[0][2]
    r12mr21 = dcm[0][1] - dcm[1][0]     
    r23pr32 = dcm[1][2] + dcm[2][1]
    r31pr13 = dcm[2][0] + dcm[0][2]
    r12pr21 = dcm[0][1] + dcm[1][0]


    if q1_den > q2_den and q1_den > q3_den and q1_den > q4_den:
        den = np.sqrt(q1_den)
        q0 = 0.5 * den
        q1 = 0.5 * r23mr32 / den
        q2 = 0.5 * r31mr13 / den
        q3 = 0.5 * r12mr21 / den
    elif q2_den > q1_den and q2_den > q3_den and q2_den > q4_den:
        den = np.sqrt(q2_den)
        q0 = 0.5 * r23mr32 / den
        q1 = 0.5 * den
        q2 = 0.5 * r12pr21 / den
        q3 = 0.5 * r31pr13 / den
    elif q3_den > q1_den and q3_den > q2_den and q3_den > q4_den:
        den = np.sqrt(q3_den)
        q0 = 0.5 * r31mr13 / den
        q1 = 0.5 * r12pr21 / den
        q2 = 0.5 * den
        q3 = 0.5 * r23pr32 / den
    elif q4_den > q1_den and q4_den > q2_den and q4_den > q3_den:
        den = np.sqrt(q4_den)
        q0 = 0.5 * r12mr21 / den
        q1 = 0.5 * r31pr13 / den
        q2 = 0.5 * r23pr32 / den
        q3 = 0.5 * den

    return np.array([q0,q1,q2,q3])


def dcmFromQuaternion(quat):
    q0 = quat[0]
    q1 = quat[1]
    q2 = quat[2]
    q3 = quat[3]

    r11 = q0 * q0 + q1 * q1 - q2 * q2 - q3 * q3
    r12 = 2 * (q1 * q2 + q0 * q3)
    r13 = 2 * (q1 * q3 - q0 * q2)
    
    r21 = 2 * (q1 * q2 - q0 * q3)    
    r22 = q0 * q0 - q1 * q1 + q2 * q2 - q3 * q3
    r23 = 2 * (q2 * q3 + q0 * q1 )

    r31 = 2 * (q1 * q3 + q0 * q2)
    r32 = 2 * (q2 * q3 - q0 * q1)
    r33 = q0 * q0 - q1 * q1 - q2 * q2 + q3 * q3

    return np.array([[ r11, r12,  r13 ],
                     [ r21, r22,  r23 ],
                     [ r31, r32,  r33 ]])
    
    


In [20]:
rads = deg2rad([15,30,75])
print(f"Euler {rads}\n")

R = dcmFromEulerXYZ(rads)
print(f"Euler {R}\n")

q = quaternionFromDCM(R)
print(f"\nquaternion:  {q}")


x = np.array([0.5, 0.2, 0.8])
y = np.matmul(R,x)
print(f"\nRotation DCM: {x}  ->  {y}")

qx = np.append(0,x)
qy = quaternionMult(q, quaternionMult(qx, quaternionInv(q)))
print(f"\nRotation Quaternion: {qx}  ->  {qy}")

# So.. y ~= qy  If you look closely at the results
#               the first element is basically zero (-5.55111512e-17 )
#               while the imaginary parts are almost identical 


# back calculat 

R1 = dcmFromQuaternion(q)
print(f"\nDCM from Quaternion:\n{q} \n DCM:\n{R1}")



Euler [0.26179939 0.52359878 1.30899694]

Euler [[ 0.22414387  0.8365163  -0.5       ]
 [-0.89951905  0.375       0.22414387]
 [ 0.375       0.39951905  0.8365163 ]]


quaternion:  [ 0.78033009 -0.05618622  0.28033009  0.55618622]

Rotation DCM: [0.5 0.2 0.8]  ->  [-0.12062481 -0.19544443  0.93661685]

Rotation Quaternion: [0.  0.5 0.2 0.8]  ->  [-5.55111512e-17 -1.20624805e-01 -1.95444432e-01  9.36616854e-01]

DCM from Quaternion:
[ 0.78033009 -0.05618622  0.28033009  0.55618622] 
 DCM:
[[ 0.22414387  0.8365163  -0.5       ]
 [-0.89951905  0.375       0.22414387]
 [ 0.375       0.39951905  0.8365163 ]]
